In [ ]:
%load_ext autoreload

In [ ]:
%autoreload 2

# An example of a Low Liquidity Trade

In [ ]:
import requests
import s3fs
import pandas as pd
import re
import humps
import sqlalchemy as sa
from datetime import timedelta, datetime
import datetime as dt
import pendulum


import yfinance as yf

from prefect import Client, Task, Flow, task, Parameter
from prefect.schedules import IntervalSchedule
from prefect.environments.storage import Docker

# Homemade
from extractMOCData import CONFIG as cfg
from extractMOCData.moc_data import TsxMocData

from normalize.ticker_symbols import MapTickerSymbols
from addFeatures.daily import DailyData 

from mocIO.read_moc import get_moc_file_lst
#from extractMOCData import read_moc 

import etl_moc

In [ ]:
# tsxMocData = TsxMocData()

# moc_df = tsxMocData.scrape_moc_data()

## Load Raw Data

In [ ]:
moc_file_lst = get_moc_file_lst.run("tsx-moc")

In [ ]:
data_flpth = f"s3://{moc_file_lst[0]}"
data_flpth

In [ ]:
tsx_moc_df = pd.read_csv(data_flpth, parse_dates=["moc_date"], na_filter=False)
tsx_moc_df.loc[tsx_moc_df["Symbol"]=="", "Symbol"] = "NA"

In [ ]:
tsx_moc_df[tsx_moc_df["Symbol"].isna()==True]

## Create a key table
Map TSX symbols to Yahoo Symbols

In [ ]:
yhooMap =  MapTickerSymbols()
moc_key_df = yhooMap.run(tsx_moc_df)

## Get daily features
(from yahoo)

In [ ]:
# Get EOD features
eod_df = etl_moc.get_eod_data.run(moc_key_df)

## Get indtraday features
(from yahoo)

In [ ]:
intraday_df = etl_moc.get_1min_ohlc.run(moc_key_df)

In [ ]:
intraday_df.info()

In [ ]:
#intraday_df['date'] =intraday_df["moc_date"].dt.date
#ts.dt.strftime('%H:%M:%S').between('00:00:00','12:00:00')

## Create MOC table

In [ ]:
moc_df = etl_moc.build_moc_data.run(intraday_df, eod_df)

## Write to Db

In [ ]:
# 1. Eod
index_clmn_lst = ["moc_date", "yahoo_symbol"]
num_rows_ins = etl_moc.df_to_db.run(intraday_df, tbl_name="intraday_prices", idx_clmn_lst=index_clmn_lst)

# 6. Write to db
num_rows_ins = etl_moc.df_to_db.run(eod_df, tbl_name="eod", idx_clmn_lst=index_clmn_lst)

# 7. Write to db
num_rows_ins = etl_moc.df_to_db.run(moc_df, tbl_name="daily_moc", idx_clmn_lst=index_clmn_lst)

In [ ]:
scrape_tsxmoc_fl.storage = Docker(dockerfile="/home/ilivni/MOC/Dockerfile")
scrape_tsxmoc_fl.register(project_name=project_nm)

In [ ]:
secret = get_secret_value_response['SecretString']

In [ ]:
secret

In [ ]:
# Use this code snippet in your app.
# If you need more information about configurations or implementing the sample code, visit the AWS docs:
# https://aws.amazon.com/developers/getting-started/ruby/

# Use this code snippet in your app.
# If you need more information about configurations or implementing the sample code, visit the AWS docs:   
# https://aws.amazon.com/developers/getting-started/python/

import boto3
import base64
from botocore.exceptions import ClientError


def get_secret():

    secret_name = "moc-pg-db"
    region_name = "us-east-2"

    # Create a Secrets Manager client
    session = boto3.session.Session()
    client = session.client(
        service_name='secretsmanager',
        region_name=region_name
    )

    # In this sample we only handle the specific exceptions for the 'GetSecretValue' API.
    # See https://docs.aws.amazon.com/secretsmanager/latest/apireference/API_GetSecretValue.html
    # We rethrow the exception by default.

    try:
        get_secret_value_response = client.get_secret_value(
            SecretId=secret_name
        )
    except ClientError as e:
        if e.response['Error']['Code'] == 'DecryptionFailureException':
            # Secrets Manager can't decrypt the protected secret text using the provided KMS key.
            # Deal with the exception here, and/or rethrow at your discretion.
            raise e
        elif e.response['Error']['Code'] == 'InternalServiceErrorException':
            # An error occurred on the server side.
            # Deal with the exception here, and/or rethrow at your discretion.
            raise e
        elif e.response['Error']['Code'] == 'InvalidParameterException':
            # You provided an invalid value for a parameter.
            # Deal with the exception here, and/or rethrow at your discretion.
            raise e
        elif e.response['Error']['Code'] == 'InvalidRequestException':
            # You provided a parameter value that is not valid for the current state of the resource.
            # Deal with the exception here, and/or rethrow at your discretion.
            raise e
        elif e.response['Error']['Code'] == 'ResourceNotFoundException':
            # We can't find the resource that you asked for.
            # Deal with the exception here, and/or rethrow at your discretion.
            raise e
    else:
        # Decrypts secret using the associated KMS CMK.
        # Depending on whether the secret is a string or binary, one of these fields will be populated.
        if 'SecretString' in get_secret_value_response:
            secret = get_secret_value_response['SecretString']
        else:
            decoded_binary_secret = base64.b64decode(get_secret_value_response['SecretBinary'])
            
    # Your code goes here. 

In [ ]:
Mschedule = IntervalSchedule(
    start_date=datetime.utcnow() + timedelta(seconds=1),
    interval=timedelta(minutes=1),
)
schedule

In [ ]:
moc_df["Symbol"] =moc_df["Symbol"].fillna("NA")

In [ ]:
moc_df[moc_df["Symbol"].isnull()]

In [ ]:
moc_df.to_json(orient="records")